**Imports**

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

**Pre-Processing**

In [5]:
data = pd.read_csv('../data/WildfireData.csv')
data.head()

,Unnamed: 0,Unnamed: 0.1,fire_name,fire_size,fire_size_class,stat_cause_descr,latitude,longitude,state,disc_clean_date,...,Wind_cont,Hum_pre_30,Hum_pre_15,Hum_pre_7,Hum_cont,Prec_pre_30,Prec_pre_15,Prec_pre_7,Prec_cont,remoteness
0,0,0,NaN,10.0,C,Missing/Undefined,18.105072,-66.753044,PR,2/11/2007,...,3.250413,78.216590,76.793750,76.381579,78.724370,0.0,0.0,0.0,0.0,0.017923
1,1,1,NaN,3.0,B,Arson,35.038330,-87.610000,TN,12/11/2006,...,2.122320,70.840000,65.858911,55.505882,81.682678,59.8,8.4,0.0,86.8,0.184355
2,2,2,NaN,60.0,C,Arson,34.947800,-88.722500,MS,2/29/2004,...,3.369050,75.531629,75.868613,76.812834,65.063800,168.8,42.2,18.1,124.5,0.194544
3,3,3,WNA 1,1.0,B,Debris Burning,39.641400,-119.308300,NV,6/6/2005,...,0.000000,44.778429,37.140811,35.353846,0.000000,10.4,7.2,0.0,0.0,0.487447
4,4,4,NaN,2.0,B,Miscellaneous,30.700600,-90.591400,LA,9/22/1999,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,0.214633


In [6]:
data = data[['fire_size', 'remoteness', 'putout_time', 'stat_cause_descr']]
data.head()

,fire_size,remoteness,putout_time,stat_cause_descr
0,10.0,0.017923,NaN,Missing/Undefined
1,3.0,0.184355,NaN,Arson
2,60.0,0.194544,NaN,Arson
3,1.0,0.487447,0 days 00:00:00.000000000,Debris Burning
4,2.0,0.214633,NaN,Miscellaneous


In [7]:
data = data.dropna()
data['putout_time'] = data['putout_time'].str.split(' ').str[0]
data.head()

,fire_size,remoteness,putout_time,stat_cause_descr
3,1.0,0.487447,0,Debris Burning
9,1.0,0.291683,0,Miscellaneous
11,37.0,0.191923,0,Arson
14,3.0,0.123518,0,Missing/Undefined
22,2.0,0.054988,0,Miscellaneous


In [8]:
data.stat_cause_descr.unique()

array(['Debris Burning', 'Miscellaneous', 'Arson', 'Missing/Undefined',
       'Lightning', 'Equipment Use', 'Campfire', 'Structure', 'Children',
       'Smoking', 'Powerline', 'Railroad', 'Fireworks'], dtype=object)

In [9]:
data['stat_cause_descr'] = data['stat_cause_descr'].map({
    'Debris Burning': 0,
    'Arson': 1,
    'Lightning': 2,
    'Equipment Use': 3,
    'Campfire': 4,
    'Structure': 5,
    'Children': 5,
    'Smoking': 5,
    'Powerline': 5,
    'Railroad': 5,
    'Fireworks': 5,
    'Missing/Undefined': 5,
    'Miscellaneous': 5
})
data.stat_cause_descr.unique()
data.head()

,fire_size,remoteness,putout_time,stat_cause_descr
3,1.0,0.487447,0,0
9,1.0,0.291683,0,5
11,37.0,0.191923,0,1
14,3.0,0.123518,0,5
22,2.0,0.054988,0,5


In [19]:
from sklearn.model_selection import train_test_split
X = data.drop('stat_cause_descr', axis=1).astype('float')
y = data.stat_cause_descr.astype('float')
print(X.head())
print(y.head())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

    fire_size  remoteness  putout_time
3         1.0    0.487447          0.0
9         1.0    0.291683          0.0
11       37.0    0.191923          0.0
14        3.0    0.123518          0.0
22        2.0    0.054988          0.0
3     0.0
9     5.0
11    1.0
14    5.0
22    5.0
Name: stat_cause_descr, dtype: float64


In [20]:
X_train.shape

(24729, 3)

In [21]:
y_train.shape

(24729,)

In [22]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [23]:
y_train.shape

(24729, 6)

**NN Model**

In [24]:
model = Sequential()
model.add(Dense(32, input_dim=(3), activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(6, activation='softmax'))

In [25]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=64, verbose=1)

Epoch 1/100
387/387 [==============================] - 2s 3ms/step - loss: 80.9891 - accuracy: 0.2770 - val_loss: 25.2122 - val_accuracy: 0.2984
Epoch 2/100
387/387 [==============================] - 1s 2ms/step - loss: 4.4688 - accuracy: 0.3069 - val_loss: 1.5959 - val_accuracy: 0.2489
Epoch 3/100
387/387 [==============================] - 1s 2ms/step - loss: 1.6041 - accuracy: 0.2607 - val_loss: 1.5266 - val_accuracy: 0.3926
Epoch 4/100
387/387 [==============================] - 1s 2ms/step - loss: 1.5383 - accuracy: 0.3778 - val_loss: 1.4797 - val_accuracy: 0.4036
Epoch 5/100
387/387 [==============================] - 1s 2ms/step - loss: 1.4999 - accuracy: 0.3839 - val_loss: 1.4475 - val_accuracy: 0.4043
Epoch 6/100
387/387 [==============================] - 1s 2ms/step - loss: 1.4783 - accuracy: 0.3973 - val_loss: 1.4336 - val_accuracy: 0.4130
Epoch 7/100
387/387 [==============================] - 1s 3ms/step - loss: 1.4627 - accuracy: 0.4003 - val_loss: 1.4057 - val_accuracy: 0.43

Epoch 58/100
387/387 [==============================] - 1s 3ms/step - loss: 1.3883 - accuracy: 0.4558 - val_loss: 1.4303 - val_accuracy: 0.4360
Epoch 59/100
387/387 [==============================] - 1s 3ms/step - loss: 1.4499 - accuracy: 0.4276 - val_loss: 1.3450 - val_accuracy: 0.4676
Epoch 60/100
387/387 [==============================] - 1s 3ms/step - loss: 1.4201 - accuracy: 0.4444 - val_loss: 1.3619 - val_accuracy: 0.4629
Epoch 61/100
387/387 [==============================] - 1s 3ms/step - loss: 1.3877 - accuracy: 0.4530 - val_loss: 1.3464 - val_accuracy: 0.4698
Epoch 62/100
387/387 [==============================] - 1s 4ms/step - loss: 1.3792 - accuracy: 0.4633 - val_loss: 1.3418 - val_accuracy: 0.4691
Epoch 63/100
387/387 [==============================] - 1s 4ms/step - loss: 1.3792 - accuracy: 0.4614 - val_loss: 1.3428 - val_accuracy: 0.4691
Epoch 64/100
387/387 [==============================] - 1s 3ms/step - loss: 1.3872 - accuracy: 0.4608 - val_loss: 1.3525 - val_accuracy:

In [17]:
# test model
test_data = {
    'fire_size': [37.0],
    'remoteness': [0.191923],
    'putout_time': [0]	
}
test_data = pd.DataFrame(test_data)
test_data

,fire_size,remoteness,putout_time
0,37.0,0.191923,0


In [ ]:
result = model.predict(test_data)
result

In [ ]:
# find class
final = 0
for i in range(6):
    if (result[0][i] > final):
        final = result[0][i]
    # print(final)
final = np.where(result[0] == final)
print(final[0][0])
    

In [ ]:
# save model
model.save("../Weights/cause.h5")

In [ ]:
# add cause to predicted fires dataset
predicted_data = pd.read_csv('../data/PredictedData.csv')

In [ ]:
predicted_data = predicted_data.dropna()
predicted_data['putout_time'] = predicted_data['putout_time'].str.split(' ').str[0]
test_data = predicted_data[['fire_size', 'remoteness', 'putout_time']].astype('float')
results = model.predict(test_data)

**XGBoost Classifier**

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [ ]:
xgb_model = XGBClassifier(
        silent=False, 
        scale_pos_weight=1,
        learning_rate=0.01,  
        colsample_bytree = 1,
        subsample = 0.8,
        objective='multi:softprob', 
        n_estimators=1000, 
        reg_alpha = 0.3,
        max_depth=5, 
        gamma=1)
xgb_model.fit(X_train, y_train)

In [ ]:
y_pred = xgb_model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [ ]:
accuracy = round(accuracy_score(y_test, predictions), 2)
accuracy

In [ ]:
result = xgb_model.predict(test_data)
result[0]

In [ ]:
print(X_test.tail())
print(y_test.tail())

In [ ]:
# test model
test_data1 = {
    'fire_size': [5.0],
    'remoteness': [0.250806],
    'putout_time': [2.0]	
}
test_data1 = pd.DataFrame(test_data1)
test_data1

In [ ]:
result1 = xgb_model.predict(test_data1)
result1[0].astype('int')